In [40]:
#pip install currencyapicom
import pandas as pd
import configparser
import currencyapicom


In [26]:
# Crear un objeto ConfigParser
config = configparser.ConfigParser()
# Leer el archivo de configuración
config.read('config.ini')
# Obtener el token de AccuWeather
CURRENCY_API_KEY = config['API_KEYS']['CURRENCY_API_KEY']

In [27]:
client = currencyapicom.Client(CURRENCY_API_KEY)

In [32]:
days = ['2022-01-01']
for day in days:    
    result = client.historical(day)
    print(result)

{'meta': {'last_updated_at': '2022-01-01T23:59:59Z'}, 'data': {'AED': {'code': 'AED', 'value': 3.67267}, 'AFN': {'code': 'AFN', 'value': 103.6321}, 'ALL': {'code': 'ALL', 'value': 105.93177}, 'AMD': {'code': 'AMD', 'value': 479.01204}, 'AOA': {'code': 'AOA', 'value': 561.3327}, 'ARS': {'code': 'ARS', 'value': 102.68228}, 'AUD': {'code': 'AUD', 'value': 1.37532}, 'AWG': {'code': 'AWG', 'value': 1.78004}, 'AZN': {'code': 'AZN', 'value': 1.69304}, 'BBD': {'code': 'BBD', 'value': 1.99814}, 'BDT': {'code': 'BDT', 'value': 85.62223}, 'BGN': {'code': 'BGN', 'value': 1.71934}, 'BHD': {'code': 'BHD', 'value': 0.37701}, 'BIF': {'code': 'BIF', 'value': 1989.14937}, 'BMD': {'code': 'BMD', 'value': 1.00003}, 'BND': {'code': 'BND', 'value': 1.34862}, 'BOB': {'code': 'BOB', 'value': 6.83016}, 'BRL': {'code': 'BRL', 'value': 5.57043}, 'BSD': {'code': 'BSD', 'value': 1.00003}, 'BTC': {'code': 'BTC', 'value': 0.0216}, 'BTN': {'code': 'BTN', 'value': 74.31136}, 'BWP': {'code': 'BWP', 'value': 11.74793}, 

In [35]:
# Convert the 'data' dictionary into a DataFrame for easier handling
df = pd.DataFrame(result) # Transpose to have currencies as rows
df.head()

,meta,data
last_updated_at,2022-01-01T23:59:59Z,NaN
AED,NaN,"{'code': 'AED', 'value': 3.67267}"
AFN,NaN,"{'code': 'AFN', 'value': 103.6321}"
ALL,NaN,"{'code': 'ALL', 'value': 105.93177}"
AMD,NaN,"{'code': 'AMD', 'value': 479.01204}"


In [36]:
# Eliminar la fila con 'last_updated_at' directamente
df = df.drop(index='last_updated_at', errors='ignore')
# Crear la columna 'last_updated_at' con el valor constante
df['last_updated_at'] = day
# Crear la columna 'money' a partir del índice y restablecer el índice para convertir las monedas en una columna
df = df.reset_index().rename(columns={'index': 'money'})
df.head()

,money,meta,data,last_updated_at
0,AED,NaN,"{'code': 'AED', 'value': 3.67267}",2022-01-01
1,AFN,NaN,"{'code': 'AFN', 'value': 103.6321}",2022-01-01
2,ALL,NaN,"{'code': 'ALL', 'value': 105.93177}",2022-01-01
3,AMD,NaN,"{'code': 'AMD', 'value': 479.01204}",2022-01-01
4,AOA,NaN,"{'code': 'AOA', 'value': 561.3327}",2022-01-01


In [37]:
# Extraer el valor del diccionario en la columna 'data' y colocarlo en la columna 'value'
df['value'] = df['data'].apply(lambda x: x['value'] if pd.notnull(x) else None)
df.head() 

,money,meta,data,last_updated_at,value
0,AED,NaN,"{'code': 'AED', 'value': 3.67267}",2022-01-01,3.67267
1,AFN,NaN,"{'code': 'AFN', 'value': 103.6321}",2022-01-01,103.63210
2,ALL,NaN,"{'code': 'ALL', 'value': 105.93177}",2022-01-01,105.93177
3,AMD,NaN,"{'code': 'AMD', 'value': 479.01204}",2022-01-01,479.01204
4,AOA,NaN,"{'code': 'AOA', 'value': 561.3327}",2022-01-01,561.33270


In [39]:
# Reordenar las columnas si es necesario (opcional)
df = df[['money', 'last_updated_at', 'value']]

# Mostrar las primeras filas del DataFrame resultante
df

,money,last_updated_at,value
0,AED,2022-01-01,3.67267
1,AFN,2022-01-01,103.63210
2,ALL,2022-01-01,105.93177
3,AMD,2022-01-01,479.01204
4,AOA,2022-01-01,561.33270
...,...,...,...
144,XPF,2022-01-01,104.63208
145,XRP,2022-01-01,1.20790
146,YER,2022-01-01,249.95497
147,ZAR,2022-01-01,15.92997


In [41]:
# Crear la carpeta si no existe
path_target = './data/dolares.csv'
os.makedirs(os.path.dirname(path_target), exist_ok=True)
df.to_csv(path_target)